In [2]:
# IMPORTAÇÃO DE BIBLIOTECAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# CONFIGURAÇÃO DE VISUALIZAÇÃO
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Define o caminho base de onde o script está
# Usando Path.cwd() ou o caminho relativo
PASTA_DATA = Path('../Data')

# CAMINHOS LOCAIS
ARQUIVO_DRE_ANUAL = PASTA_DATA / 'balanco_magazine luiza (tratado) - dem. resultados anual.csv'
ARQUIVO_BP_ANUAL = PASTA_DATA / 'balanco_magazine luiza (tratado) - balanço patrimonial anual.csv'
ARQUIVO_MACRO = PASTA_DATA / 'Tabela 3 e 4. Taxas de crescimento do PIB, desemprego, inflação, NUCI e câmbio - 2010-2020.csv'

# TRATAMENTO
def preparar_dre_anual(df_dre):
    df_dre_t = df_dre.set_index(df_dre.columns[0]).T.reset_index()
    df_dre_t.rename(columns={'index': 'Ano'}, inplace=True)
    
    # CONTAS CHAVE NA DRE
    NOME_RECEITA_LIQUIDA = 'Receita Líquida de Vendas e/ou Serviços'
    NOME_RESULTADO_BRUTO = 'Resultado Bruto'
    NOME_LUCRO_LIQUIDO = 'Lucro/Prejuízo do Período'


    # SELECIONAR E RENOMEAR COLUNAS
    df_dre_final = df_dre_t[['Ano', NOME_RECEITA_LIQUIDA, NOME_RESULTADO_BRUTO, NOME_LUCRO_LIQUIDO]].copy()
    df_dre_final.columns = ['Ano', 'Receita_Liq', 'Resultado_Bruto', 'Lucro_Liquido']


    # CONVERTER ANO E DADOS PARA NUMÉRICO
    df_dre_final['Ano'] = pd.to_numeric(df_dre_final['Ano'], errors='coerce', downcast='integer')
    for col in ['Receita_Liq', 'Resultado_Bruto', 'Lucro_Liquido']:
        df_dre_final[col] = pd.to_numeric(df_dre_final[col], errors='coerce')
        
    # IMPUTA 0 NAS CONTAS CONTÁBEIS ANTES DO CÁLCULO PARA EVITAR NAN NO CÁLCULO DE CRESCIMENTO
    df_dre_final[['Receita_Liq', 'Resultado_Bruto', 'Lucro_Liquido']] = df_dre_final[['Receita_Liq', 'Resultado_Bruto', 'Lucro_Liquido']].fillna(0)
        
    # CALCULAR A VARIÁVEL ALVO: CRESCIMENTO ANUAL DA RECEITA
    df_dre_final['MGLU3_Cresc_AA'] = df_dre_final['Receita_Liq'].pct_change() * 100
    
    # SUBSTITUI O NAN GERADO PELO PCT_CHANGE NO PRIMEIRO ANO POR 0
    df_dre_final['MGLU3_Cresc_AA'].fillna(0, inplace=True)
    
    return df_dre_final.dropna(subset=['Ano']).sort_values('Ano')


def preparar_bp_anual(df_bp):
    # TRANSPOR E RENOMEAR A COLUNA DE ANO (DATA)
    df_bp_t = df_bp.set_index(df_bp.columns[0]).T.reset_index()
    df_bp_t.rename(columns={'index': 'Ano'}, inplace=True)
    
    # CONTAS CHAVE NO BP
    NOME_ATIVO_TOTAL = 'Ativo Total'
    NOME_ATIVO_CIRCULANTE = 'Ativo Circulante'
    NOME_PASSIVO_CIRCULANTE = 'Passivo Circulante'
    NOME_PATRIMONIO_LIQUIDO = 'Patrimônio Líquido'


    # SELECIONAR E RENOMEAR COLUNAS
    df_bp_final = df_bp_t[['Ano', NOME_ATIVO_TOTAL, NOME_ATIVO_CIRCULANTE, NOME_PASSIVO_CIRCULANTE, NOME_PATRIMONIO_LIQUIDO]].copy()
    df_bp_final.columns = ['Ano', 'Ativo_Total', 'Ativo_Circulante', 'Passivo_Circulante', 'Patrimonio_Liquido']
    df_bp_final['Ano'] = pd.to_numeric(df_bp_final['Ano'], errors='coerce', downcast='integer')
    for col in ['Ativo_Total', 'Ativo_Circulante', 'Passivo_Circulante', 'Patrimonio_Liquido']:
        df_bp_final[col] = pd.to_numeric(df_bp_final[col], errors='coerce')
        
    # IMPUTA 0 NAS CONTAS CONTÁBEIS ANTES DO CÁLCULO PARA EVITAR NAN NO CÁLCULO DE INDICADORES
    cols_bp = ['Ativo_Total', 'Ativo_Circulante', 'Passivo_Circulante', 'Patrimonio_Liquido']
    df_bp_final[cols_bp] = df_bp_final[cols_bp].fillna(0)


    return df_bp_final.dropna(subset=['Ano']).sort_values('Ano')



def preparar_macro(df_macro):
    # SELEÇÃO DE VARIÁVEIS MACRO DE INTERESSE
    df_macro_rel = df_macro[df_macro['Discriminação'].isin(['PIB', 'Consumo das Famílias'])].set_index('Discriminação')
    df_macro_t = df_macro_rel.T.reset_index().rename(columns={'index': 'Ano'})
    
    # SELEÇÃO E RENOMEAMENTO
    df_macro_final = df_macro_t[['Ano', 'PIB', 'Consumo das Famílias']].copy()
    df_macro_final.columns = ['Ano', 'PIB_Var_Real', 'Consumo_Familia_Var_Real']
    df_macro_final['Ano'] = pd.to_numeric(df_macro_final['Ano'], errors='coerce', downcast='integer')
    for col in ['PIB_Var_Real', 'Consumo_Familia_Var_Real']:
        df_macro_final[col] = pd.to_numeric(df_macro_final[col], errors='coerce')
        
    # SUBSTITUI NAN NAS VARIÁVEIS MACRO POR 0
    cols_macro = ['PIB_Var_Real', 'Consumo_Familia_Var_Real']
    df_macro_final[cols_macro] = df_macro_final[cols_macro].fillna(0)
        
    return df_macro_final.dropna(subset=['Ano'])

# EXECUÇÃO DO PRE-PROCESSAMENTO
df_dre_raw = pd.read_csv(ARQUIVO_DRE_ANUAL)
df_bp_raw = pd.read_csv(ARQUIVO_BP_ANUAL)
df_macro_raw = pd.read_csv(ARQUIVO_MACRO, decimal=',') 
df_dre_processado = preparar_dre_anual(df_dre_raw)
df_bp_processado = preparar_bp_anual(df_bp_raw)
df_macro_processado = preparar_macro(df_macro_raw)

print("--- Arquivo Original (DRE ANUAL) ---")
print(df_dre_raw.head())
print("------------------------------------------")
print("--- Arquivo Original (balanço patrimonial anual) ---")
print(df_bp_raw.head())
print("------------------------------------------")
print("--- Arquivo Original (Indicadores Econômicos) ---")
print(df_macro_raw.head())
print("------------------------------------------")


print("Dados de DRE, BP e Macro processados com sucesso.")


# MERGE DRE E BP PARA CALCULAR INDICADORES INTERNOS
df_internos = pd.merge(df_dre_processado, df_bp_processado, on='Ano', how='inner')


# GARANTIR O TRATAMENTO DE DIVISÃO POR ZERO.
with np.errstate(divide='ignore', invalid='ignore'):
    df_internos['Margem_Bruta'] = np.divide(df_internos['Resultado_Bruto'], df_internos['Receita_Liq'], 
                                            out=np.zeros_like(df_internos['Resultado_Bruto'], dtype=float), 
                                            where=df_internos['Receita_Liq']!=0) * 100
    df_internos['ROA'] = np.divide(df_internos['Lucro_Liquido'], df_internos['Ativo_Total'],
                                   out=np.zeros_like(df_internos['Lucro_Liquido'], dtype=float),
                                   where=df_internos['Ativo_Total']!=0) * 100
    df_internos['ROE'] = np.divide(df_internos['Lucro_Liquido'], df_internos['Patrimonio_Liquido'],
                                   out=np.zeros_like(df_internos['Lucro_Liquido'], dtype=float),
                                   where=df_internos['Patrimonio_Liquido']!=0) * 100
    df_internos['Giro_Ativo'] = np.divide(df_internos['Receita_Liq'], df_internos['Ativo_Total'],
                                          out=np.zeros_like(df_internos['Receita_Liq'], dtype=float),
                                          where=df_internos['Ativo_Total']!=0)
    df_internos['Liquidez_Corrente'] = np.divide(df_internos['Ativo_Circulante'], df_internos['Passivo_Circulante'],
                                                  out=np.zeros_like(df_internos['Ativo_Circulante'], dtype=float),
                                                  where=df_internos['Passivo_Circulante']!=0)


# SELECIONAR COLUNAS RELEVANTES PARA O MERGE
colunas_modelo = [
    'Ano', 
    'MGLU3_Cresc_AA',
    'Margem_Bruta',
    'ROE',
    'Giro_Ativo',
    'Liquidez_Corrente'
]
df_modelo_mglu = df_internos[colunas_modelo].copy()


# RELACIONAR DADOS DA MGLU3 COM DADOS MACRO
df_modelo_final = pd.merge(df_modelo_mglu, df_macro_processado, on='Ano', how='inner')

# PREENCHE QUAISQUER NANS REMANESCENTES COM 0 (DEVIDO A ARREDONDAMENTOS OU DIVISÕES NÃO COBERTAS).
df_modelo_final.fillna(0, inplace=True) 

# CONVERTE QUAISQUER VALORES INFINITOS (INF) QUE POSSAM TER SOBRADO DE DIVISÕES POR ZERO PARA 0.
df_modelo_final.replace([np.inf, -np.inf], 0, inplace=True)

# RESULTADO FINAL (DATASET PRONTO PARA IA)

print("\n=======================================================================")
print("DATASET FINAL PRONTO PARA PROJEÇÃO (IA)")
print("=======================================================================")
print(f"Número de Linhas (Anos) após imputação zero: {df_modelo_final.shape[0]}")
print(f"Colunas (Variáveis): {df_modelo_final.shape[1]}")
print("\nDataFrame Final (df_modelo_final):")
print(df_modelo_final)


# SALVAR O ARQUIVO FINAL
ARQUIVO_SAIDA = PASTA_DATA / 'mglu3_dataset_projecao_final.csv'
df_modelo_final.to_csv(ARQUIVO_SAIDA, index=False)
print(f"\nArquivo final salvo em: '{ARQUIVO_SAIDA}'")



--- Arquivo Original (DRE ANUAL) ---
                               Ano Vigente      2019      2018      2017     2016     2015     2014      2013     2012     2011
0  Receita Líquida de Vendas e/ou Serviços  19886309  15590444  11984250  9508746  8978259  9779385  22172922  7665112  6419371
1     Custo de Bens e/ou Serviços Vendidos -14332350 -11053022  -8378239 -6586130 -6399630 -7086909 -15247125 -5146597 -4275084
2                          Resultado Bruto   5553961   4537422   3606011  2922615  2578629  2692476   6925797  2518515  2144287
3                      Despesas Com Vendas  -3444112  -2747447  -2119953 -1776258 -1720799 -1746258  -4365789 -1581049 -1270971
4        Despesas Gerais e Administrativas  -1188562   -759833   -679085  -615545  -584280  -556882  -1429211  -481965  -441566
------------------------------------------
--- Arquivo Original (balanço patrimonial anual) ---
                     Ano Vigente      2019      2018      2017      2016      2015      2014      

/tmp/ipykernel_21230/1918700808.py:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dre_final['MGLU3_Cresc_AA'].fillna(0, inplace=True)
